# Exploring Credit Loan Data

## 1. Introduction

In credit business, company has to make decision about loans approval based on the applicant's profile. Company shall able to identify if the loan applicants is likely to repay the loan or not. The consequences are:

- If the applicants is able to repay the loan, missing this kind of applicants may result in business loss
- If the applicants is not able to repay the loan, accepting the loan from this applicants is also result in business loss.

Lending Club is peer-to-peer lending company, which people in needs can apply for loan, and some can become the investor by providing loan funding and later enjoy the interest as the profit.

While data growth is significant in this era, we can utilize it to become more useful. In this occasion, writer is trying to utilize *Lending Club Data* to predict creditworthiness of future loan applicants by developing a Machine Learning Model to predict if specific loan (defined by the loan structure and applicants background) will defaulted or not.

## 2. Exploring the Dataset

In [1]:
import pandas as pd
import numpy as np
from modules.data_exploration import DataExploration
from modules.data_preprocess import LoanDataPreprocess
from joblib import dump, load

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 75)

data_preprocess = LoanDataPreprocess().fit()
loan_df = pd.read_csv('dataset\lc_2010-2015.csv', dtype={'desc': 'str', 'verification_status_joint': 'str'})
loan_df = data_preprocess.transform(loan_df)
loan_df.head()
    

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501.00,1296599.00,5000.00,5000.00,4975.00,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.00,Verified,2011-12-01,Fully Paid,n,Borrower added on 12/22/11 > I need to upgrade...,credit_card,Computer,860xx,AZ,27.65,0.00,1985-01-01,1.00,NaN,NaN,3.00,0.00,13648.00,83.70,9.00,f,0.00,0.00,5861.07,5831.78,5000.00,861.07,0.00,0.00,0.00,2015-01-01,171.62,NaT,2016-01-01,0.00,NaN,1.00,INDIVIDUAL,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430.00,1314167.00,2500.00,2500.00,2500.00,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.00,Source Verified,2011-12-01,Charged Off,n,Borrower added on 12/22/11 > I plan to use thi...,car,bike,309xx,GA,1.00,0.00,1999-04-01,5.00,NaN,NaN,3.00,0.00,1687.00,9.40,4.00,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,2013-04-01,119.66,NaT,2013-09-01,0.00,NaN,1.00,INDIVIDUAL,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175.00,1313524.00,2400.00,2400.00,2400.00,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.00,Not Verified,2011-12-01,Fully Paid,n,NaN,small_business,real estate business,606xx,IL,8.72,0.00,2001-11-01,2.00,NaN,NaN,2.00,0.00,2956.00,98.50,10.00,f,0.00,0.00,3003.65,3003.65,2400.00,603.65,0.00,0.00,0.00,2014-06-01,649.91,NaT,2016-01-01,0.00,NaN,1.00,INDIVIDUAL,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863.00,1277178.00,10000.00,10000.00,10000.00,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.00,Source Verified,2011-12-01,Fully Paid,n,Borrower added on 12/21/11 > to pay for proper...,other,personel,917xx,CA,20.00,0.00,1996-02-01,1.00,35.00,NaN,10.00,0.00,5598.00,21.00,37.00,f,0.00,0.00,12226.30,12226.30,10000.00,2209.33,16.97,0.00,0.00,2015-01-01,357.48,NaT,2015-01-01,0.00,NaN,1.00,INDIVIDUAL,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358.00,1311748.00,3000.00,3000.00,3000.00,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.00,Source Verified,2011-12-01,Current,n,Borrower added on 12/21/11 > I plan on combini...,other,Personal,972xx,OR,17.94,0.00,1996-01-01,0.00,38.00,NaN,15.00,0.00,27783.00,53.90,38.00,f,766.90,766.90,3242.17,3242.17,2233.10,1009.07,0.00,0.00,0.00,2016-01-01,67.79,2016-02-01,2016-01-01,0.00,NaN,1.00,INDIVIDUAL,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.1. Data fields containing 'Zero' or 'Missing' records

I will give a more detailed review for data fields that has *non-null* less than total records of 466,285. They must have *zeroes* or *missing records (NaN)* and needed to be identified, since each value may have different meaning (for example, *0* in datafield *open_acc* means no current credit account opened while *missing/NaN* records clearly explain that no information recorded).
After this review, we can consider how to deal with this condition for preparation of machine learning features.

In [11]:
applicant_features = [
    'member_id', 'emp_title', 'emp_length', 'home_ownership',
    'annual_inc', 'verification_status', 'dti', 'delinq_2yrs',
    'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
    'mths_since_last_record', 'open_acc', 'revol_bal', 'revol_util',
    'total_acc', 'collections_12_mths_ex_med', 'mths_since_last_major_derog',
    'policy_code', 'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 
    'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
    'il_util', 'open_rv_12m', 'open_rv_24m', 'all_util', 'total_rev_hi_lim', 
    'inq_fi', 'total_cu_tl', 'inq_last_12m', 'pub_rec'
    ]

loan_features = [
    'id', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade',
    'sub_grade', 'pymnt_plan', 'desc', 'purpose', 'title',
    'zip_code', 'addr_state', 'policy_code', 'application_type',
    'total_pymnt', 'total_pymnt_inv'
    ]
post_origin_features = [
    'funded_amnt', 'funded_amnt_inv', 'issue_d', 'loan_status', 
    'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_rec_prncp',
    'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
    'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d'
]

#### **Missing Records**

It is best to categorize the missing data based on the type of missing. Most commonly known types of missing data are as follows:

- **Structurally missing data**, is the data that is missing for a logical reason, since the reason caused them to not exist. For example from our data, missing data of `annual_inc_joint` is a structural missing if the loan's `application_type` is `INDIVIDUAL` loan rather than `JOINT` loan.
- **Missing Completely at Random (MCAR)**, is the data that is missing with no reasonable connection or pattern to another data. For example, `ann_inc` is very specific record of each observation, hence missing this record is categorized as MCAR. Since we can do nothing to overcome this missing value, the most possible action is to drop the observations if the features is crucial to avoid any possible biases.
- **Missing at Random (MAR)**, is the data that is missing but the connection or pattern can be predicted (e.g. using extrapolation) or extracted from combination of other features (e.g. substraction, summation, etc.). For example, if an observation has a missing `int_rate` record, we can possibly calculate it by using `loan_amnt` and `total_pymnt`.
- **Missing not at Random (MNAR)**, is the data where the reason or mechanism for why the data is missing is known. For example, missing loan `desc` is known because the loan applicant was not inputting it to the system.

##### *Applicant Profile* and *Loan Profile* Features

*Applicant Profile* and *Loan Profile* is our main features for developing the machine learning model. Action to overcome the missing features will be explained accordingly.

Let's start by *Applicant Profile*.

In [12]:
data_exploration = DataExploration(loan_df)
data_exploration.show_nans_or_zeroes('nans', applicant_features).sort_values('Nans Count', ascending=False)

,Nans Count,Nans Percentage (%),Data Types
dti_joint,886870,99.94,float64
annual_inc_joint,886868,99.94,float64
verification_status_joint,886868,99.94,category
il_util,868762,97.90,float64
mths_since_rcnt_il,866569,97.65,float64
total_cu_tl,866007,97.59,float64
inq_last_12m,866007,97.59,float64
all_util,866007,97.59,float64
open_rv_24m,866007,97.59,float64
open_rv_12m,866007,97.59,float64


In [20]:
data_exploration.show_nans_or_zeroes('nans', applicant_features).sort_values('Nans Count', ascending=False).index

Index(['dti_joint', 'annual_inc_joint', 'verification_status_joint', 'il_util',
       'mths_since_rcnt_il', 'total_cu_tl', 'inq_last_12m', 'all_util',
       'open_rv_24m', 'open_rv_12m', 'open_acc_6m', 'total_bal_il', 'inq_fi',
       'open_il_12m', 'open_il_24m', 'mths_since_last_record',
       'mths_since_last_major_derog', 'mths_since_last_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'emp_title', 'emp_length',
       'revol_util', 'collections_12_mths_ex_med', 'pub_rec', 'acc_now_delinq',
       'total_acc', 'open_acc', 'inq_last_6mths', 'earliest_cr_line',
       'delinq_2yrs', 'annual_inc', 'policy_code', 'revol_bal', 'dti',
       'verification_status', 'home_ownership', 'member_id'],
      dtype='object')

1. Structurally Missing: `total_rev_hi_lim`, `tot_coll_amnt`, `tot_curr_bal`, `revol_util`, `collections_12_mths_ex_med`, `inq_last_6mths`, `delinq_2yrs`, `earliest_cr_line`, `total_acc`, `open_acc`, `acc_now_delinq`, and most of the features that has a high percentage of missing data ($>$ 50%). Note on missing percentage of `_joint` features, we can easily validate that most of the loan is `INDIVIDUAL` loan. We will drop the observations with `JOINT` type loan due to limited data compared to `INDIVIDUAL` loan hence limiting the scope of the model to covers only `INDIVIDUAL` loan. Low variance features (missing data $>$ 50%) will be dropped, and other missing will be replaced as `0`.

2. MNAR: `emp_title`. High cardinal data won't be used for machine learning features.

3. MCAR: `annual_inc`, `emp_length`, `earliest_cr_line`. Observations with misisng `annual_inc` will be dropped. For `emp_length`, we will one-hot encode this and also provide *out-of-vocabulary* dummy variables for this missing data. `earlist_cr_line` will be utilized for feature extraction.

>More on *Structurally Missing* data
>1. `mths_since_last_record` is linked with `pub_rec`. Missing records is proportional with *zeroes* records in `pub_rec`. For this features. Replacing missing records in `mths_since_last_record` with *zeroes* while `pub_rec` is zero is valid. But for the rest of the missing value when `pub_rec` is not zero, we don't know the most appropriate value to impute this records. I choose to drop this feature since `pub_rec` is enough to represents public records history of applicant. 
>2. Filling out a lot of missing values for `mths_since_last_derog` is not preferable so better to drop this feature.
>3. `mths_since_last_delinq` is linked with `delinq_2yrs` this way: records with `mths_since_last_delinq` <= 24 will prove `delinq_2yrs` at least 1, > 24 then `delinq_2yrs` must be 0. Some insights:
    - if `delinq_2yrs` = 0, we still can't justify `mths_since_last_delinq` value will be 0, because value > 24 is also correct. Number of records of `delinq_2yrs` = 0 and `mths_since_last_delinq` = N/A is 403,618 records, which is pretty high.
    - There are 496 records of missing `mths_since_last_delinq` but `delinq_2yrs` records exist. 
    - Because of high missing values portion and the way to justify this value is vague and pretty difficult, best option is to drop this feature. `delinq_2yrs` is enough to represents applicants recent history of deliquency.
>4. `tot_coll_amt`, `total_rev_hi_lim` and `tot_cur_bal` has a high missing value covering 2.3% `Default` and 23.5% `Charged Off` loan status. Dropping rows that are having these columns not recorded is very regretted and dropping these features may let us misses  important correlations for our classifier.

For *Loan Profile*:

In [16]:
data_exploration.show_nans_or_zeroes('nans', loan_features).sort_values('Nans Count', ascending=False)

,Nans Count,Nans Percentage (%),Data Types
desc,761351,85.80,object
title,152,0.02,object
purpose,0,0.00,category
total_pymnt,0,0.00,float64
application_type,0,0.00,category
policy_code,0,0.00,float64
addr_state,0,0.00,object
zip_code,0,0.00,object
id,0,0.00,float64
loan_amnt,0,0.00,float64


Type of missing data is MNAR: `desc`, `title`. These features won't be used for machine learning features.

### **Cardinality of data fields containing *Objects* datatype**

In [15]:
obj_columns = loan_df.select_dtypes(include='object').columns

for column in obj_columns:
    print('Column: ', column, ', Unique Values: ', len(loan_df[column].unique()))

Column:  emp_title , Unique Values:  289145
Column:  desc , Unique Values:  124454
Column:  title , Unique Values:  61446
Column:  zip_code , Unique Values:  935
Column:  addr_state , Unique Values:  51


Data fields with `object` datatype, which have high cardinality, won't be used as machine learning features. Nevertheless, statistics related to those datafields will be shown for exploratory analysis to gather valuable insight. Later we will use *word clouds* as a representation of these observations.

### **Takeaway**

Based on explained above, below list consists of features that will be dropped later when preparing dataset for machine learning model.

In [ ]:
cols_to_drop = [
    'dti_joint', 'annual_inc_joint', 'verification_status_joint', 'il_util',
    'mths_since_rcnt_il', 'total_cu_tl', 'inq_last_12m', 'all_util',
    'open_rv_24m', 'open_rv_12m', 'open_acc_6m', 'total_bal_il', 'inq_fi',
    'open_il_12m', 'open_il_24m', 'mths_since_last_record',
    'mths_since_last_major_derog', 'mths_since_last_delinq', 'earliest_cr_line'
    ]

cols_impute_zero = [
    'revol_util', 'collections_12_mths_ex_med', 'pub_rec', 'acc_now_delinq',
    'total_acc', 'open_acc', 'inq_last_6mths',
    'delinq_2yrs', 
    ]

## 2. Exploratory Data Analytics

### 2.1. Categorical Data

Analyzing categorical data gives us general understanding of the Lending Club dataset, especially about the loan status that implies general outcome of this Lending Club credit business. Summary statistics for our categorical data is shown as below:

In [18]:
loan_df.select_dtypes(include=['object', 'category']).describe(include='all')

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,initial_list_status,application_type,verification_status_joint
count,887379,887379,887379,835917,842554,887379,887379,887379,887379,126028,887379,887227,887379,887379,887379,887379,511
unique,2,7,35,289144,11,6,3,10,2,124453,14,61445,935,51,2,2,3
top,36 months,B,B3,Teacher,10+ years,MORTGAGE,Source Verified,Current,n,,debt_consolidation,Debt consolidation,945xx,CA,f,INDIVIDUAL,Not Verified
freq,621125,254535,56323,13807,291569,443557,329558,601779,887369,247,524215,414352,9770,129517,456848,886868,283


#### **Loan Status**

Feature `loan_status` is the possible features as a target of supervised learning to predict whether particular loan is good or bad. This feature records is available after the loan is originated to the applicant.

- On-going loan has status: `Current`, `Late`, and `In Grace Period`
- Finished loan has status either `Fully Paid`, `Charged Off`, and `Default`. 

Most of the loan listed in our dataset is on-going loan (`loan_status` = `Current`). However, we can identify some loan that faces some difficulties in paying-off debt on the due date: `Late` and `In Grace Period`. 

To make an easier representation of `loan_status`, we will create a new feature to label the loan as `Good Loan` and `Bad Loan`.

- `Bad Loan` consists of: `Charged Off`, `Default`, `In Grace Period`, and `Late`.
- `Good Loan` consists of: `Fully Paid`.
- `Current` loan possibly may be a `Good Loan`, since up to the data recorded the loan seems to not having paying difficulties.
 
Assumming these loan to be a `Good Loan` completely is too risky since it is highly subjective. On the other hand, ignoring these loans completely (which covers more than half of the dataset!) is also not a wise move. To overcome this, we will consider `Current Loan` as a `Good Loan` only if the loan is close to be fully paid, specifically when the `total_rec_prncp` $\geq$ `specific` portion of `loan_amnt` (recall that `total_rec_prncp` is *Principal received to date*).

Let's see the distribution of portion of loan paid of `Default` and `Charged Off` status:

Points observed:

- Imbalanced sample between `Good Loan` and `Bad Loan`. This probably would lead to model with low recall in predicting potentially `Bad Loan` (false negative occurs more frequent). For credit business, it is more preferable to avoid defaulted loan rather than taking a risk for maximizing loan numbers then hoping succesfull loan will outnumber (in profit) defaulted loan. Action should be taken.

In [ ]:
loan_df['loan_status'] = \
    pd.Categorical(loan_df['loan_status'], categories=['Fully Paid', 'Charged Off'])

loan_df.drop(
    loan_df[loan_df['loan_status'].isna()].index,
    inplace=True)

#### **Loan Grade by Loan Status**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax= plt.subplots(1, 2, figsize=(18,6))

for ax_, cat in zip(ax, ['grade', 'sub_grade']):
    ax_.grid(visible=True, axis='y')
    sns.countplot(x=cat, data=loan_df, hue='loan_status', ax=ax_)
    ax_.set_title(f'{cat} by loan_status')
    ax_.set_axisbelow(True)

ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)

plt.show()

#### **Employment Length by Loan Status**

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 8))
sns.countplot(x='emp_length', data=loan_df, hue='loan_status', ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.grid(visible=True, axis='y')
ax.set_axisbelow(True)
ax.set_title('Employment Length by loan_status')


#### **Home Ownership by Loan Status**

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 8))
sns.countplot(x='home_ownership', data=loan_df, hue='loan_status', ax=ax)
ax.grid(visible=True, axis='y')
ax.set_axisbelow(True)
ax.set_title('Home Ownership by loan_status')

#### **Loan Status by Purpose**

In [ ]:
%matplotlib inline

loan_purpose_sorted = \
    loan_df['purpose'].value_counts(ascending=False).index.tolist()
fig, ax = plt.subplots(figsize=(12, 8))
sns.countplot(y='purpose', data=loan_df, hue='loan_status', ax=ax, order=loan_purpose_sorted)
ax.grid(visible=True, axis='x')
ax.set_axisbelow(True)
ax.set_title('Loan Status by Purpose')
ax.legend(loc=4)

In [ ]:
loan_purpose_sorted = \
    loan_df['title'].value_counts(ascending=False).index.tolist()[:5]

loan_purpose_subset = \
    loan_df[loan_df['title'].isin(loan_purpose_sorted)]

(ggplot(loan_purpose_subset, aes(x='title'))
+ geom_bar(
    aes(fill='title'),
    show_legend=False,
    )
+ geom_text(
    stat='count',
    mapping=aes(
        label=after_stat('count'),
        size=50
        ),
    show_legend=False,
    nudge_y = 4000
    )
+ scale_x_discrete(limits=loan_purpose_sorted[::-1])
+ labs(
    x='Purpose Label',
    y='Count', 
    title=f'5 Most Frequent Loan Purposes. Total records: {len(loan_purpose_subset):,}'
    )
+ coord_flip() 
+ theme_linedraw()
)

### 2.2. Numerical Data

In [ ]:
loan_df.describe()

In [ ]:
# subset the numerical features
cols_subset = (
    ['loan_status']
    + loan_df.select_dtypes(exclude=['object', 'category']).columns.tolist()
) 

# remove columns that unrelated to machine learning features
remove_cols = [
    'funded_amnt', 
    'funded_amnt_inv', 
    'issue_d', 
    'initial_list_status', 
    'out_prncp', 
    'out_prncp_inv', 
    'total_pymnt', 
    'total_pymnt_inv', 
    'total_rec_prncp', 
    'total_rec_int', 
    'total_rec_late_fee', 
    'recoveries', 
    'collection_recovery_fee', 
    'last_pymnt_d', 
    'last_pymnt_amnt', 
    'next_pymnt_d', 
    'last_credit_pull_d', 
    'id',
    'member_id',
    'policy_code',
    'earliest_cr_line'
]

# DataFrame for correlation matrix
loan_df_subset = loan_df[cols_subset].copy()
loan_df_subset.drop(loan_df_subset.columns[loan_df_subset.columns.isin(remove_cols)], axis=1, inplace=True)



In [ ]:
import phik
from phik import resources, report
from scipy import stats

# phi-k correlation matrix
loan_df_subset_phik_corr = loan_df_subset.phik_matrix()['loan_status']

# point biserial correlation
pbis_corr, pbis_p = [], []
loan_df_subset.loan_status = loan_df_subset.loan_status.map({'Fully Paid': 0, 'Charged Off': 1})
for var_ in loan_df_subset.columns:
    try:
        pbis_corr.append(
            stats.pointbiserialr(loan_df_subset[var_], 
            loan_df_subset['loan_status'])[0]
            )
        pbis_p.append(
            stats.pointbiserialr(loan_df_subset[var_], 
            loan_df_subset['loan_status'])[1]
            )
    except (ValueError, AttributeError): # Error caused by NaN records
        pbis_corr.append(
            stats.pointbiserialr(loan_df_subset.loc[~loan_df_subset[var_].isna(), var_], 
            loan_df_subset.loc[~loan_df_subset[var_].isna(), 'loan_status'])[0]
            )
        pbis_p.append(
            stats.pointbiserialr(loan_df_subset.loc[~loan_df_subset[var_].isna(), var_], 
            loan_df_subset.loc[~loan_df_subset[var_].isna(), 'loan_status'])[1]
            )
loan_df_subset_pbis_corr = pd.Series(pbis_corr, index=loan_df_subset.columns)
loan_df_subset_pbis_p = pd.Series(pbis_p, index=loan_df_subset.columns)

#combining subset
loan_df_subset_corr = pd.concat([loan_df_subset_phik_corr,
                                loan_df_subset_pbis_corr,
                                np.abs(loan_df_subset_pbis_corr),
                                loan_df_subset_pbis_p],
                                axis=1)
loan_df_subset_corr.columns = [
    'phi-k corr.', 'point biserial corr.', 'point biserial corr. (mag.)', 'point biserial p-value'
    ]
loan_df_subset_corr.drop('loan_status').sort_values('point biserial corr. (mag.)', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

plot_corr_df = \
    loan_df_subset_corr.drop('loan_status')\
                       .reset_index()\
                       .melt(id_vars=['index'], 
                             value_vars=loan_df_subset_corr.columns[[0, 2]])
plot_order = loan_df_subset_corr.drop('loan_status').sort_values('point biserial corr. (mag.)', ascending=False).index

sns.barplot(y='index', x='value', hue='variable', data = plot_corr_df, order=plot_order, ax=ax)
ax.grid(visible=True, axis='x')
ax.set_axisbelow(True)
ax.set_xlabel('Correlation Coefficient, loan_status')
ax.set_title('Correlation Coefficient of Numerical Features')

High difference in correlation coefficient magnitude occurs for features `revol_util`, `tot_cur_bal`, `ann_inc`, and `total_rev_hi_lim`. Note that phi-k correlation is very sensitive to outliers. Let's discover each of the variables' distribution.

In [ ]:
# bin_width = \
#     2 * (loan_df['loan_amnt'].quantile(0.75) - loan_df['loan_amnt'].quantile(0.25)) / (len(loan_df['loan_amnt'])**(1/3))

x = plot_order

fig, ax= plt.subplots(len(x), 2, figsize=(18, 6 * len(x)))

for x, axes in zip(x, ax):
    sns.histplot(data=loan_df, x=x, bins=30, hue='loan_status', ax=axes[0], alpha=0.5)
    sns.boxplot(data=loan_df, x=x, y='loan_status', ax=axes[1])
    axes[0].grid(visible=True, axis='y')
    axes[0].set_axisbelow(True)
    axes[0].set_title(f'Histogram, {x}')
    axes[1].set_title(f'Central Tendency, {x}')

From above plots, points observed:

- Variables from above to below is sorted by the highest to lowest correlation coefficient.
- Interest rate (`int_rate`) is the most significant feature among others. 
- Each variables has a highly different value scales, needed to be standardized.
- From above to below, the distribution between `Fully Paid` and `Charged Off` class is becoming less different. It may inform that the more different the distribution between classes, the more that variable may be useful and importance to the model as classification features. 
- We should focus on several features that has skewed (right-skewed, specifically) distributions and have a high number of outliers.
- it is clear that only `dti` has a more normal-distribution than others, identified by the histogram and well-balanced boxplot (centered median and balanced whisker for class `Charged Off`, however class `Fully Paid` is less Gaussian). Other than that, especially `revol_util`, `tot_cur_bal`, `ann_inc`, `total_rev_hi_lim` and other variables that has correlation coefficient close to zero, has a lot of outliers.
- For features that majority records zeroes (for example `pub_rec`, `acc_now_delinq`, etc.), the outliers is (probably) not exactly an anomaly. Hence if we simply discard it we may lose important information that may reduce our data representativeness. In the next process we will conduct more detailed outlier identification for each of the features.

Next, we will inspect numerical features that are highly correlated with other features. Heatmap below shows Spearman's correlation coefficient, where the order of the columns and rows are determined by a clustering algorithm. I don't want to focus on the clustering algorithm here, but the motivation is to make the correlation plot more tidy. I will utilize a high-level seaborn clustermap to do this.

In [ ]:
loan_df_subset_corr = loan_df_subset.corr(method='spearman')
sns.clustermap(loan_df_subset_corr, vmin=-1, vmax=1, center=0, cmap='vlag', annot=True, fmt='.2f', annot_kws={'size': 7})

There is a pair of features that show high correlation: `loan_amnt` and `installment`. According to the explanation, it is clear because installment is a monthly payment based on loan amount (+ *interest*) divided by number of payment (i.e. loan `term`). So we can just use `installment` to cover both `loan_amnt` and `term`. Note that we can utilize this high correlation to identify outliers, which will explore more later at the next chapter. 

Closing note for our numerical features: less gaussian distribution, a lot of outliers, and a less linearly related variables make our feature analysis by utilizing correlation coefficient is not properly suitable for our dataset. Indeed we still can  observe generally which variables may be potential for our model features and which one may be discarded to avoid overfitting.

Later, more robust methods for feature selection will be conducted. Feature transformation to make the distribution more Gaussian will also be conducted.

## 3. Summary

We already got some understanding for a credit business and the motivation behind developing machine learning classifier, specifically for *Lending Club Peer-to-peer Loan*, although the provided dataset may set constraint for our model here and there.

Then, we discovered that our dataset has several variables that uninsightful for machine learning modeling. Also, the characteristic of our data is not ideal enough to implement statistical method directly hence need some cleaning and transformations. 

Lastly, we can conclude below information for our next activity: **Data preparation and feature engineering**.

1. Subsetting the data
    - Discard the features related to post-origanted loan to avoid leaking the information to our model that may infer the class.
    - Choose on of the features that highly correlated with each other (between `loan_amnt` and `installment`)
    - Discard the features with zero records
    - Discard the high cardinality columns (`emp_title`, `url`, `desc`, `title`, `zip_code`, `addr_state`)
    - Subset the `loan_status` row to include only `Fully Paid` and `Charged Off` class

2. Cleaning the data
    - Some discrete numeric features have inconsistent data types that need to be corrected
    - The dataset has a feature of ordinal category: `employment length`
    - Some features have missing data to be handled
    - Some features have a lot of outliers to be handled

3. Dataset Preparation and Feature Engineering
    - Handle the imbalanced sample
    - Feature transformation for skewed data
    - Encoding categorical features
    - Feature selection